# Welcome to HAM Tool Box !

This ipython notebook aims to help you to do your first steps with HAM. 

We will covered in this tutorial how to: 
* [set](#set) up an HAM analysis using the different options available.
* shows the different [queries](#query) that you can use to fetch information you want.
* explore what all ham [objects](#object)  have to offer !
*  [compare](#compare) several genomes throught their HOGS.
* run hogvis (single hog visualisation)
* run treeprofile (species tree with annotated node with evolutionary events)



<a id='set'></a>
## SET-UP THE HAM

### Required import

In [1]:
#  This is the HAM package
import ham

#  OPTIONAL: only if you want to have the logger information printed
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

### First, we need a species tree and an orthoXML !

In [2]:
import os, inspect # This is just to get the exmaple file from your ham package installation

#  Select a nwk file as a taxonomy reference
nwk_path = os.path.dirname(inspect.getfile(ham)) + "/../tests/data/simpleEx.nwk"
#  And extract the newick tree as a string
tree_str = ham.utils.get_newick_string(nwk_path, type="nwk")

# Then you select your favorite orthoXML file

orthoxml_path = os.path.dirname(inspect.getfile(ham)) + "/../tests/data/simpleEx.orthoxml"


### Let's feed HAM with some HOGs and a tree.

In [3]:
# ham.HAM is th/e main object that contained all information and functionalities.
ham_analysis = ham.HAM(tree_str, orthoxml_path, use_internal_name=True)

2017-05-11 22:06:17,270 ham.ham      INFO     Build taxonomy: completed.
2017-05-11 22:06:17,272 ham.parsers  INFO     Species HUMAN created. 
2017-05-11 22:06:17,274 ham.parsers  INFO     Species PANTR created. 
2017-05-11 22:06:17,275 ham.parsers  INFO     Species CANFA created. 
2017-05-11 22:06:17,276 ham.parsers  INFO     Species MOUSE created. 
2017-05-11 22:06:17,277 ham.parsers  INFO     Species RATNO created. 
2017-05-11 22:06:17,278 ham.parsers  INFO     Species XENTR created. 
2017-05-11 22:06:17,281 ham.ham      INFO     Parse Orthoxml: 3 top level hogs and 19 extant genes extract.
2017-05-11 22:06:17,282 ham.ham      INFO     Set up HAM analysis: ready to go with 3 hogs founded within 6 species.


### Alternative options: filtering and taxonomic range naming !

#### Taxonomic range naming
If you don't want to use the internal names defined by the newick specie tree (or if it's have support values as internal names) you can use the automatic naming of HAM (by concatenation of node child names).


In [4]:
# the use_internal_name attribute is doing the job
ham_analysis_no_name = ham.HAM(tree_str, orthoxml_path, use_internal_name=False)

# In the previous section we use the internal name of the newick tree:
print("Ancestral genomes name using internal names:")
for ag in ham_analysis.taxonomy.internal_nodes:
    print("\t- {}".format(ag.name))
    
# Here we use the artificial internal names built by ham:
print("Ancestral genomes name using artificial ham internal names:")
for ag in ham_analysis_no_name.taxonomy.internal_nodes:
    print("\t- {}".format(ag.name))

#  In case you are using use_internal_name=False, the following function returns an ascii representation
#  of the newick tree with  articial names. This can be used even before instanciating the HAM object which
#  can be usefull to avoid stupid error in script.

print(ham.utils.previsualize_taxonomy(tree_str))


2017-05-11 22:06:17,292 ham.ham      INFO     Build taxonomy: completed.
2017-05-11 22:06:17,293 ham.parsers  INFO     Species HUMAN created. 
2017-05-11 22:06:17,294 ham.parsers  INFO     Species PANTR created. 
2017-05-11 22:06:17,295 ham.parsers  INFO     Species CANFA created. 
2017-05-11 22:06:17,296 ham.parsers  INFO     Species MOUSE created. 
2017-05-11 22:06:17,297 ham.parsers  INFO     Species RATNO created. 
2017-05-11 22:06:17,297 ham.parsers  INFO     Species XENTR created. 
2017-05-11 22:06:17,300 ham.ham      INFO     Parse Orthoxml: 3 top level hogs and 19 extant genes extract.
2017-05-11 22:06:17,300 ham.ham      INFO     Set up HAM analysis: ready to go with 3 hogs founded within 6 species.


Ancestral genomes name using internal names:
	- Rodents
	- Primates
	- Euarchontoglires
	- Vertebrata
	- Mammalia
Ancestral genomes name using artificial ham internal names:
	- MOUSE/RATNO
	- HUMAN/PANTR
	- XENTR/HUMAN/PANTR/MOUSE/RATNO/CANFA
	- HUMAN/PANTR/MOUSE/RATNO
	- HUMAN/PANTR/MOUSE/RATNO/CANFA

                                   /-XENTR
                                  |
                                  |                                                               /-HUMAN
-XENTR/HUMAN/PANTR/MOUSE/RATNO/CANFA                                                   /HUMAN/PANTR
                                  |                                                   |           \-PANTR
                                  |                             /HUMAN/PANTR/MOUSE/RATNO
                                  |                            |                      |           /-MOUSE
                                   \HUMAN/PANTR/MOUSE/RATNO/CANFA                      \MOUSE/RATNO
          

#### Pre Parsing filtering
If you have a large orthoXML file or you only want parse information of interest you can use the filter option.

In [5]:
# First you instanciate an empty ParserFilter object
filter_ham = ham.ParserFilter()

# Then you can add to the filter which hog you want using the following function (those 3 examples are doing the same)
filter_ham.add_hogs_via_hogId([2]) # by its toplevel hog id 
filter_ham.add_hogs_via_GeneIntId([2]) # by a gene unique id that belong to the hog of interest 
filter_ham.add_hogs_via_GeneExtId(["HUMAN2"]) # by a gene external id that belong to the hog of interest 

# Finnaly, you set up the HAM object as you learned with the filter_object attribute
ham_analysis_filter = ham.HAM(tree_str, orthoxml_path, use_internal_name=True, filter_object=filter_ham)

# If you look at the log information beneath it is differnt from the one previously printed !!

2017-05-11 22:06:17,311 ham.ham      INFO     Build taxonomy: completed.
2017-05-11 22:06:17,313 ham.ham      INFO     Filtering Indexing of Orthoxml done: 1 top level hogs and 4 extant genes will be extract.
2017-05-11 22:06:17,314 ham.parsers  INFO     Species HUMAN created. 
2017-05-11 22:06:17,315 ham.parsers  INFO     Species PANTR created. 
2017-05-11 22:06:17,316 ham.parsers  INFO     Species CANFA created. 
2017-05-11 22:06:17,316 ham.parsers  INFO     Species MOUSE created. 
2017-05-11 22:06:17,317 ham.parsers  INFO     Species RATNO created. 
2017-05-11 22:06:17,318 ham.parsers  INFO     Species XENTR created. 
2017-05-11 22:06:17,319 ham.ham      INFO     Parse Orthoxml: 1 top level hogs and 4 extant genes extract.
2017-05-11 22:06:17,320 ham.ham      INFO     Set up HAM analysis: ready to go with 1 hogs founded within 6 species.


<a id='query'></a>
## QUERIES: HOW TO TALK WITH HAM ?

We can split the queries in diffents types:
*  [ExtantGene](#xgeneq)
*  [HOG](#hogq)
*  [ExtantGenome](#xgenomeq)
*  [AncestralGenome](#ancgenomeq)
*  [Taxon](#taxq)

<a id='xgeneq'></a>
### ExtantGene queries

In [6]:
# Get a gene by its unique (orthoxml) id
gene_human3 = ham_analysis.get_gene_by_id(3)
print(gene_human3)

# Get a list of genes that matches an external id
potentential_genes_human3 = ham_analysis.get_genes_by_external_id("HUMAN3")
print(potentential_genes_human3[0])

## You should see twice the same gene printed beneath !

# You can also get all genes created as a list
list_genes = ham_analysis.get_list_extant_genes()

# or as the dictionnary (key <-> unique id and value <-> gene)
dict_genes = ham_analysis.get_dict_extant_genes()

Gene(3)
Gene(3)


<a id='hogq'></a>
### HOG queries

In [7]:
# Get a HOG by its top level unique id
HOG_3 = ham_analysis.get_hog_by_id(3)
print(HOG_3)

HOG_3 = ham_analysis.get_hog_by_gene(gene_human3)
print(HOG_3)

## You should see twice the same HOG printed beneath !

# You can also get all genes created as a list
list_toplevel_hogs = ham_analysis.get_list_top_level_hogs()

# or as the dictionnary (key <->  top level id and value <-> HOG)
dict_toplevel_hogs = ham_analysis.get_dict_top_level_hogs()

<HOG(3)>
<HOG(3)>


<a id='xgenomeq'></a>
### ExtantGenome queries

In [8]:
# Get a extant genome by its name
genome_human = ham_analysis.get_extant_genome_by_name("HUMAN")
print(genome_human)

# You can also get all extant genomes created as a list
list_genome = ham_analysis.get_list_extant_genomes()
print("List of species:")
for g in list_genome:
    print("\t- {}".format(g.name))

HUMAN
List of species:
	- HUMAN
	- XENTR
	- MOUSE
	- CANFA
	- PANTR
	- RATNO


<a id='ancgenomeq'></a>
### AncestralGenome queries

In [9]:
# Get an ancestral genome by its name
genome_rodents_1 = ham_analysis.get_ancestral_genome_by_name("Rodents")
print(genome_rodents_1)

# And in case you specified use_internal_name=False ! 
genome_rodents_2 = ham_analysis_no_name.get_ancestral_genome_by_name("MOUSE/RATNO")
print(genome_rodents_2)

# Get an ancestral genome by its the looking at the mrca of 2+ genomes
# First you get the descendant genomes as we seen above
genome_rat = ham_analysis.get_extant_genome_by_name("RATNO")
genome_mouse = ham_analysis.get_extant_genome_by_name("MOUSE")
# Then you get the corresponding mrca ancestral genomes 
genome_rodents_3 = ham_analysis.get_ancestral_genome_by_mrca_of_genome_set({genome_rat, genome_mouse})
print(genome_rodents_3)

# You can also get an ancestral genome by its taxonomy node
taxon = ham_analysis.get_taxon_by_name("Rodents") # as seen in the next section
genome_rodents_4 = ham_analysis.get_ancestral_genome_by_taxon(taxon)
print(genome_rodents_4)

# You can also get all ancestral genomes created as a list
list_genome = ham_analysis.get_list_ancestral_genomes()
print("\n List of ancestral genomes:")
for g in list_genome:
    print("\t- {}".format(g.name))
    
    
# You can also get all ancestral genomes created as a list
list_genome = ham_analysis_filter.get_list_ancestral_genomes()
print("\n Here for the filter version of HAM, the list of ancestral genomes:")
for g in list_genome:
    print("\t- {}".format(g.name))
print("'Vertebrate' is not present in the filter list because not required during the parsing")

Rodents
MOUSE/RATNO
Rodents
Rodents

 List of ancestral genomes:
	- Rodents
	- Primates
	- Euarchontoglires
	- Vertebrata
	- Mammalia

 Here for the filter version of HAM, the list of ancestral genomes:
	- Euarchontoglires
	- Mammalia
	- Rodents
	- Primates
'Vertebrate' is not present in the filter list because not required during the parsing


<a id='taxq'></a>
### Taxon queries

In [10]:
# You can get an Taxonomy node by its name
taxon = ham_analysis.get_taxon_by_name("Rodents") 
print(taxon.name)

taxon = ham_analysis.get_taxon_by_name("HUMAN") 
print(taxon.name)

Rodents
HUMAN


<a id='object'></a>
## HAM OBJECT: SHOW ME YOUR SECRETS ?

We can split the queries in diffents types:
*  [Gene/HOG](#gdef)
*  [Genome](#genomedef)
*  [Taxonomy](#taxonomydef)


<a id='gdef'></a>
### Gene/HOG

#### AbstractGene (Both ExtantGene and HOG)

In [11]:
# we select one extant gene and one hog for our demo
print("Demo| gene/hog: ")
gene2 = ham_analysis.get_gene_by_id(2)
print("\t - gene 2: ", gene2)
hog1 = ham_analysis.get_hog_by_id(1)
print("\t - hog 1: ", hog1)

Demo| gene/hog: 
	 - gene 2:  Gene(2)
	 - hog 1:  <HOG(1)>


###### The cool feature in common for both of them are:

In [12]:
# - that we can fetch the genome where they belong to
print("Demo| gene/hog genome: ")
print("\t - gene 2: ", gene2.genome)
print("\t - hog 1: ", hog1.genome)
print("\n")

Demo| gene/hog genome: 
	 - gene 2:  HUMAN
	 - hog 1:  Vertebrata




In [13]:
# - that we can get the related top level hog
print("Demo| gene/hog related top level hog: ")
print("\t - gene 2: ", gene2.get_top_level_hog())
print("\t - hog 1: {} (it  returns itself it already top level hog) ".format(hog1.get_top_level_hog()))
print("\n")

Demo| gene/hog related top level hog: 
	 - gene 2:  <HOG(2)>
	 - hog 1: <HOG(1)> (it  returns itself it already top level hog) 




In [14]:
# - that we can get the related hog at a specific level 
rodents = ham_analysis.get_ancestral_genome_by_name("Rodents")
print("Demo| gene/hog related top level hog: ")
print("\t - gene 2: ", gene2.get_at_level(rodents))
print("\t - hog 1: ", hog1.get_at_level(rodents))
print("/!\ THIS CAN RETURN MORE THAN ONE HOG /!\ ")
hog3 = ham_analysis.get_hog_by_id(3)
print("\t - hog 3: ", hog3.get_at_level(rodents))
print("\n")

Demo| gene/hog related top level hog: 
	 - gene 2:  [<HOG()>]
	 - hog 1:  [<HOG(1.M.E.R)>]
/!\ THIS CAN RETURN MORE THAN ONE HOG /!\ 
	 - hog 3:  [<HOG()>, <HOG()>]




In [15]:
# - that we can know if the abstractGene is a singleton or not
human_5 = ham_analysis.get_gene_by_id("5")
print("Demo| gene/hog singletons ? : ")
print("\t - gene 2: {}".format(gene2.is_singleton()))
print("\t - gene 5: {}".format(human_5.is_singleton()))
print("\t - HOG 1: {}".format(hog1.is_singleton()))
print("\n")

Demo| gene/hog singletons ? : 
	 - gene 2: False
	 - gene 5: True
	 - HOG 1: False




In [16]:
# - that we can get the ancestor HOG of an abstractGene at a specific level. In addition a boolean specify if a duplication occured in between the two levels.
mamm = ham_analysis.get_ancestral_genome_by_name("Mammalia")
hog3_rodents = hog3.get_at_level(rodents)[0]

print("Demo| gene ancestor hog at Mammalia level: ")
print("\t - gene 2: {}".format(gene2.search_ancestor_hog_in_ancestral_genome(mamm)))
print("\t - hog 3 at rodents: {}".format(hog3_rodents.search_ancestor_hog_in_ancestral_genome(mamm)))

Demo| gene ancestor hog at Mammalia level: 
	 - gene 2: (<HOG(2)>, False)
	 - hog 3 at rodents: (<HOG(3.M)>, True)


#### ExtantGene

In [17]:
# You can get a dictionary with all the cross references for a gene
gene2 = ham_analysis.get_gene_by_id(2)
print(gene2.get_dict_xref())

{'id': '2', 'geneId': 'HUMANg2', 'protId': 'HUMAN2'}


#### AncestralGene

###### The cool feature in for HOG are that you can:

In [18]:
# demo hog
hog1 = ham_analysis.get_hog_by_id(1)
hog3 = ham_analysis.get_hog_by_id(3)

In [19]:
# get all descendant genes
desc_genes = hog1.get_all_descendant_genes()
print(desc_genes)

[Gene(51), Gene(21), Gene(1), Gene(11), Gene(31), Gene(41)]


In [20]:
# get all descendant genes clustered by species
desc_genes_clustered = hog3.get_all_descendant_genes_clustered_by_species()
for species, genes in desc_genes_clustered.items():
    print(species, genes)

MOUSE [Gene(33), Gene(34)]
CANFA [Gene(23)]
XENTR [Gene(53)]
HUMAN [Gene(3)]
PANTR [Gene(13), Gene(14)]


In [21]:
# get all descendant level
desc_level = hog1.get_all_descendant_hog_levels()
for genome in desc_level:
    print(genome.name)

Vertebrata
Mammalia
Euarchontoglires
Primates
Rodents


In [22]:
# get all descendant hogs
desc_hog = hog1.get_all_descendant_hogs()
print(desc_hog)

[<HOG(1)>, <HOG(1.M)>, <HOG(1.M.E)>, <HOG(1.M.E.P)>, <HOG(1.M.E.R)>]


In [23]:
# visit the hog with prefix, postfix, leaf callback function
# (reading the docs at ham.abstract.HOG.visit() is required here) 

# -----------------------------------------------------------#
# -- This is an example to apply a function to each leaves --#
# -----------------------------------------------------------#
def print_and_append_leaf(current, child, list):
    list.append(child)
    print(child)
    return list

passed_object = []
return_object = hog1.visit(passed_object, function_extant_gene=print_and_append_leaf)

print("The return object (list of leaves): {}".format(return_object))

Gene(51)
Gene(21)
Gene(1)
Gene(11)
Gene(31)
Gene(41)
The return object (list of leaves): [Gene(51), Gene(21), Gene(1), Gene(11), Gene(31), Gene(41)]


In [24]:
# ------------------------------------------------------------------#
# -- This is an example to apply a function to each node (prefix) --#
# ------------------------------------------------------------------#
def print_and_append_node(current, list):
    list.append(current)
    print(current)
    return list

passed_object = []
return_object = hog1.visit(passed_object, function_prefix=print_and_append_node)

print("The return object (list of internale nodes): {}".format(return_object))

<HOG(1)>
<HOG(1.M)>
<HOG(1.M.E)>
<HOG(1.M.E.P)>
<HOG(1.M.E.R)>
The return object (list of internale nodes): [<HOG(1)>, <HOG(1.M)>, <HOG(1.M.E)>, <HOG(1.M.E.P)>, <HOG(1.M.E.R)>]


In [25]:
# -------------------------------------------------------------------------------------------#
# -- This is an example to apply a function to each node after the recursive call (prefix) --#
# -------------------------------------------------------------------------------------------#
def print_and_append_node(self, child, elem):
    elem.append(child)
    print(child)
    return elem

passed_object = []
return_object = hog1.visit(passed_object, function_postfix=print_and_append_node)

print("The return object (list of internale nodes child): {}".format(return_object))

<HOG(1.M.E.P)>
<HOG(1.M.E.R)>
<HOG(1.M.E)>
<HOG(1.M)>
The return object (list of internale nodes child): [<HOG(1.M.E.P)>, <HOG(1.M.E.R)>, <HOG(1.M.E)>, <HOG(1.M)>]


<a id='genomedef'></a>
### Genome


In [26]:
human_genome = ham_analysis.get_extant_genome_by_name("HUMAN")
rodents_genome = ham_analysis.get_ancestral_genome_by_name("Rodents")

# You can get the name
print("Get genome name:")
print(human_genome.name)
print(rodents_genome.name)
print("\n")

Get genome name:
HUMAN
Rodents




In [27]:
# You can get the related node in the taxonomy
print("Get genome taxon:")
print(human_genome.taxon)
print(rodents_genome.taxon)
print("\n")

Get genome taxon:

--HUMAN

   /-MOUSE
--|
   \-RATNO




In [28]:
# You can get the list of genes associated to this genomes
print("Get genome genes:")
print(human_genome.genes)
print(rodents_genome.genes)
print("\n")

Get genome genes:
[Gene(1), Gene(2), Gene(3), Gene(5)]
[<HOG(1.M.E.R)>, <HOG()>, <HOG()>, <HOG()>]




In [29]:
# You can get the number of genes associated to this genomes
print("Get genome genes number:")
print(human_genome.get_number_genes(singleton=True))
print(human_genome.get_number_genes(singleton=False))
print(rodents_genome.get_number_genes())
print("\n")

Get genome genes number:
4
3
4




In [30]:
for h, gs in rodents.get_ancestral_clustering().items():
    print("HOG: {} -> genes: {}".format(h,gs))

HOG: <HOG()> -> genes: [Gene(33)]
HOG: <HOG(1.M.E.R)> -> genes: [Gene(31), Gene(41)]
HOG: <HOG()> -> genes: [Gene(34)]
HOG: <HOG()> -> genes: [Gene(32)]


<a id='taxonomydef'></a>

### Taxonomy

The ham.taxonomy.Taxonomy object contains all informations about the species tree structure.

In [31]:
# it contains all the leaves nodes:
print(ham_analysis.taxonomy.leaves)
print("\n")

# and the internal node:
print(ham_analysis.taxonomy.internal_nodes)
print("\n")

# The main interest of this object is the taxonomy.tree object.
tree = ham_analysis.taxonomy.tree

# Since it's an ete3.Etree tree is contains all the built-in functionalities
for node in ham_analysis.taxonomy.tree.traverse(): # .traverse() is an ete3.etree method.
    print(node.name, node.is_leaf(), node.is_root())

{Tree node 'HUMAN' (0x10befe40), Tree node 'XENTR' (0x10befe47), Tree node 'MOUSE' (-0x7fffffffef4101af), Tree node 'CANFA' (-0x7fffffffef41018c), Tree node 'PANTR' (0x10befe55), Tree node 'RATNO' (0x10befe5c)}


{Tree node 'Rodents' (0x10befe71), Tree node 'Primates' (-0x7fffffffef4101bd), Tree node 'Euarchontoglires' (-0x7fffffffef4101c4), Tree node 'Vertebrata' (0x106e02fd), Tree node 'Mammalia' (0x10befe39)}


Vertebrata False True
XENTR True False
Mammalia False False
Euarchontoglires False False
CANFA True False
Primates False False
Rodents False False
HUMAN True False
PANTR True False
MOUSE True False
RATNO True False


<a id='compare'></a>
## COMPARE SEVERAL GENOMES

In HAM, you can compare the genome based on the evolutionary history of their genes. 

**Vertical comparison** For example you can compare the human genome with its ancestor at the level of Vertebrates. This mean that you will investigate on how the ancestral genes in the Vertebrates ancestral genome have evolved (did they stay single copy, have duplicated or been lost) to gave rise to their related descendants in the human genomes. This is not restricted to extant genomes, you could also compare the rodents with the mammals.

As show in the following example:

In [32]:
# Get the genome of interest
human = ham_analysis.get_extant_genome_by_name("HUMAN")
vertebrates = ham_analysis.get_ancestral_genome_by_name("Vertebrata")

# Instanciate the gene mapping !
vertical_human_vertebrates = ham_analysis.compare_genomes_vertically(human, vertebrates) # The order doesn't matters !

# Ask the mapper about the genes evolutionary history between the human and the vertebrates.

# The identical genes (that stay single copies) 
print("HOG at vertebrates -> descendant gene in human")
print(vertical_human_vertebrates.get_identical())
print("\n")

# The duplicated genes (that have duplicated) 
print("HOG at vertebrates -> list of descendants gene in human")
print(vertical_human_vertebrates.get_duplicated())
print("\n")

# The gained genes (that emerged in between)
print("List of human gene")
print(vertical_human_vertebrates.get_gained())
print("\n")

# The lost genes (that been lost in between) 
print("HOG at vertebrates that are lost")
print(vertical_human_vertebrates.get_lost())
print("\n")


HOG at vertebrates -> descendant gene in human
{<HOG(1)>: Gene(1)}


HOG at vertebrates -> list of descendants gene in human
{<HOG(3)>: [Gene(3)]}


List of human gene
[Gene(5), Gene(2)]


HOG at vertebrates that are lost
set()




**Lateral comparison** An other example could be to compare the rodents ancestral genome vs the primates ancestral genome through their common ancestor at the level of Euarchontoglires. In the same principle as explained in the last paragraph you are investigating on how the ancestral Euarchontoglires genes had behave to gave rise to their related descendant in both primates and rodents ancestrals genomes.

In [34]:
# Get the genome of interest
human = ham_analysis.get_extant_genome_by_name("HUMAN")
mouse = ham_analysis.get_extant_genome_by_name("RATNO")

# Instanciate the gene mapping !
lateral_human_mouse = ham_analysis.compare_genomes_lateral(human, mouse) # The order doesn't matters !

# Ask the mapper about the genes evolutionary history between the human and the rat with and the Euarchontoglires (their mrca).

# The identical genes (that stay single copies) 
print("IDENTICAL GENES")
for hogs, dict_genome_gene in lateral_human_mouse.get_identical().items():
    print("\t- HOG at Euarchontoglires {} is the ancestor of: ".format(hogs))
    for g, gene in dict_genome_gene.items():
        print("\t\t-  {} in {}".format(gene, g))
print("\n")

# The duplicated genes (that have duplicated) 
print("DUPLICATED GENES")
for hogs, dict_genome_genes in lateral_human_mouse.get_duplicated().items():
    print("\t- HOG at Euarchontoglires {} is the ancestor of: ".format(hogs))
    for g, genes in dict_genome_gene.items():
        print("\t\t-  {} in {}".format(genes, g))
print("\n")

# The gained genes (that emerged in between)
print("GAINED GENES")
for genome, gains in lateral_human_mouse.get_gained().items():
    print("\t- Genome {} have gained:".format(genome))
    for g in gains:
        print("\t\t-  {}".format(g))
print("\n")

# The lost genes (that been lost in between) 
print("LOST GENES")
for hog, genomes in lateral_human_mouse.get_lost().items():
    print("\t- HOG at Euarchontoglires {} have been lost in ".format(hog))
    for g in genomes:
        print("\t\t- {}".format(g))
print("\n")


IDENTICAL GENES
	- HOG at Euarchontoglires <HOG(1.M.E)> is the ancestor of: 
		-  Gene(41) in RATNO
		-  Gene(1) in HUMAN
	- HOG at Euarchontoglires <HOG(2.E)> is the ancestor of: 
		-  Gene(2) in HUMAN
	- HOG at Euarchontoglires <HOG(3.E.1)> is the ancestor of: 
		-  Gene(3) in HUMAN


DUPLICATED GENES


GAINED GENES
	- Genome RATNO have gained:
		-  Gene(43)
	- Genome HUMAN have gained:
		-  Gene(5)


LOST GENES
	- HOG at Euarchontoglires <HOG(2.E)> have been lost in 
		- RATNO
	- HOG at Euarchontoglires <HOG(3.E.2)> have been lost in 
		- RATNO
		- HUMAN
	- HOG at Euarchontoglires <HOG(3.E.1)> have been lost in 
		- RATNO


